In [ ]:
import netCDF4 as nc
import numpy as np

#fft_path = '/home/jim/Desktop/FFT_notebook/BPMFwFFT/examples/fft_sampling/t4_benzene'
fft_path = '/home/jim/Desktop/FFT_notebook/5.fft_sampling/1AVZ_B:C'
fft_sample_nc = '%s/fft_sample.nc'%fft_path

variable_names = nc.Dataset(fft_sample_nc, 'r').variables
rec_positions = nc.Dataset(fft_sample_nc, 'r').variables["rec_positions"][:]
lig_positions = nc.Dataset(fft_sample_nc, 'r').variables["lig_positions"][:]
lig_com = nc.Dataset(fft_sample_nc, 'r').variables["lig_com"][:]
volume = nc.Dataset(fft_sample_nc, 'r').variables["volume"][:]
nr_grid_points = nc.Dataset(fft_sample_nc, 'r').variables["nr_grid_points"][:]
exponential_sums = nc.Dataset(fft_sample_nc, 'r').variables["exponential_sums"][:]
log_of_divisors = nc.Dataset(fft_sample_nc, 'r').variables["log_of_divisors"][:]

mean_energy = nc.Dataset(fft_sample_nc, 'r').variables["mean_energy"][:]
min_energy = nc.Dataset(fft_sample_nc, 'r').variables["min_energy"][:]
resampled_energies = nc.Dataset(fft_sample_nc, 'r').variables["resampled_energies"][:]
resampled_trans_vectors = nc.Dataset(fft_sample_nc, 'r').variables["resampled_trans_vectors"][:]
print(resampled_trans_vectors[:])
print(resampled_energies[:])

#grid coordinates
origin = nc.Dataset(fft_sample_nc, 'r').variables["origin"][:]
x = nc.Dataset(fft_sample_nc, 'r').variables["x"][:]
y = nc.Dataset(fft_sample_nc, 'r').variables["y"][:]
z = nc.Dataset(fft_sample_nc, 'r').variables["z"][:]
d = nc.Dataset(fft_sample_nc, 'r').variables["d1"][:]
#print('native pose:', str(resampled_trans_vectors[0][0]))
#print('native pose:', resampled_energies[0][0])

#print('test', np.where(resampled_energies ==-1412.9091438865273))
#print('min energy pose:', resampled_trans_vectors[120][0])
#print('min energy pose:', resampled_energies[120][0])

for name in variable_names:
    print(name)

# for i, vec in enumerate(lig_com):
#     adj_vec = []
#     for x in vec:
#         adj_vec.append(x/0.5)
#     print('lig_com:%s'%i, vec, adj_vec)

# for i, vec in enumerate(resampled_trans_vectors[0]):
#     adj_vec = []
#     for x in vec:
#         adj_vec.append(x*0.5)
#     print('vec%s:'%i, str(adj_vec), str(vec))


print(lig_positions.shape)
print(resampled_energies.shape)
print(resampled_trans_vectors.shape)
print(np.amin(resampled_energies[0]))
min_index = np.where(resampled_energies[0] == resampled_energies[0].min())
print(min_index)
print(float(resampled_energies[0][min_index]))
print(resampled_trans_vectors[0][min_index])

In [ ]:
nc.close()

In [ ]:
from grids import RecGrid
from grids import LigGrid

rec_prmtop = "/home/jim/Desktop/FFT_notebook/1.amber/1AVZ_B:C/receptor.prmtop"
lj_sigma_scal_fact = 0.8
rec_inpcrd = "/home/jim/Desktop/FFT_notebook/2.minimize/1AVZ_B:C/receptor.inpcrd"

bsite_file = None
grid_nc_file = "/home/jim/Desktop/FFT_notebook/4.receptor_grid/1AVZ_B:C/grid.nc"

lig_prmtop = "/home/jim/Desktop/FFT_notebook/1.amber/1AVZ_B:C/ligand.prmtop"
lig_inpcrd = "/home/jim/Desktop/FFT_notebook/2.minimize/1AVZ_B:C/ligand.inpcrd"



def _create_rec_grid(rec_prmtop, lj_sigma_scal_fact, rec_inpcrd, bsite_file, grid_nc_file):
    rec_grid = RecGrid(rec_prmtop, lj_sigma_scal_fact, rec_inpcrd, bsite_file, 
                        grid_nc_file, new_calculation=False)
    return rec_grid

def _create_lig_grid(lig_prmtop, lj_sigma_scal_fact, lig_inpcrd, rec_grid):
    lig_grid = LigGrid(lig_prmtop, lj_sigma_scal_fact, lig_inpcrd, rec_grid)
    return lig_grid

rec_grid = _create_rec_grid(rec_prmtop, lj_sigma_scal_fact, rec_inpcrd, bsite_file, grid_nc_file)

lig_grid = _create_lig_grid(lig_prmtop, lj_sigma_scal_fact, lig_inpcrd, rec_grid)



    

In [ ]:
print(len(lig_grid.get_charges()['CHARGE_E_UNIT']))
#print(lig_grid.get_crd())
print(resampled_trans_vectors[0])
trans_lig_crd = []
for i, crd in enumerate(lig_grid.get_crd()):
    shifted_crds = []
    shifted_crds.append(crd[0]+(resampled_trans_vectors[0][0][0]*.5))
    shifted_crds.append(crd[1]+(resampled_trans_vectors[0][0][1]*.5))
    shifted_crds.append(crd[2]+(resampled_trans_vectors[0][0][2]*.5))
    trans_lig_crd.append(shifted_crds)

trans_lig_crd_array = np.array(trans_lig_crd)
print(trans_lig_crd[0])
print(lig_grid.get_crd()[0])
#print(rec_grid.direct_energy(trans_lig_crd_array, lig_grid.get_charges()))


In [ ]:
import IO

pdb_filename = '/home/jim/Desktop/FFT_notebook/ligand_test2.pdb'

lig_prmtop = "/home/jim/Desktop/FFT_notebook/1.amber/1AVZ_B:C/ligand.prmtop"



IO.write_pdb(lig_prmtop, lig_grid.get_crd(), pdb_filename, "w")

IO.write_pdb(lig_prmtop, trans_lig_crd_array, '/home/jim/Desktop/FFT_notebook/ligand_test2.pdb', "w")

In [ ]:
rotation_nc = '/home/jim/Desktop/FFT_notebook/3.ligand_rand_rot/1AVZ_B:C/rotation.nc'

rotation_variables = nc.Dataset(rotation_nc, 'r').variables["positions"]

for i, rotation in enumerate(rotation_variables):
    IO.write_pdb(lig_prmtop, rotation_variables[i], '/home/jim/Desktop/FFT_notebook/7.analysis/1AVZ_B:C/ligand_rotation%s.pdb'%i, "w")

print(rotation_variables.shape)
#IO.write_pdb(lig_prmtop, rotation_variables, '/home/jim/Desktop/FFT_notebook/7.analysis/1AVZ_B:C/ligand_rotation%s.pdb'%i, "w")

In [ ]:
# find the average translation vector between the origin centered structure and the initial translation
# this is mostly just a test, it's not needed for anything

native_translations = []

for i in range(0,889):
    native_translations.append([lig_grid.get_crd()[i][0]-rotation_variables[0][i][0], lig_grid.get_crd()[i][1]-rotation_variables[0][i][1], lig_grid.get_crd()[i][2]-rotation_variables[0][i][2]])
    #print(lig_grid.get_crd()[i][0]-rotation_variables[0][i][0], lig_grid.get_crd()[i][1]-rotation_variables[0][i][1], lig_grid.get_crd()[i][2]-rotation_variables[0][i][2])
native_trans_array = np.array(native_translations)

print(np.average(native_trans_array, axis=0))

In [ ]:
def _get_molecule_center_of_mass(self):
    """
    return the center of mass of self._crd
    """
    center_of_mass = np.zeros([3], dtype=float)
    masses = self._prmtop["MASS"]
    for atom_ind in range(len(self._crd)):
        center_of_mass += masses[atom_ind] * self._crd[atom_ind]
    total_mass = masses.sum()
    if total_mass == 0:
        raise RuntimeError("zero total mass")
    return center_of_mass / total_mass

In [ ]:
print(lig_grid.get_initial_com())

In [ ]:
IO.write_pdb(lig_prmtop, lig_positions[1], "/home/jim/Desktop/FFT_notebook/ligand_test5_rot2.pdb", "w")

In [ ]:
for i, position in enumerate(lig_positions):
    IO.write_pdb(lig_prmtop, rotation_variables[i], '/home/jim/Desktop/FFT_notebook/7.analysis/1AVZ_B:C/ligand_positions%s.pdb'%i, "w")


In [ ]:
# get list of lowest energy translations
#def get_min_energy_translations:
import IO
trans_tuples = []
for i, rotation in enumerate(resampled_energies):
    min_index = np.where(rotation == rotation.min())
    trans_tuples.append((rotation[min_index], resampled_trans_vectors[i][min_index]))

#print(trans_tuples[0][1][0][0])

trans_lig_crd = []
for i, position in enumerate(lig_positions):
    shifted_crds = []
    for j, crd in enumerate(position):
        shifted_crds.append([crd[0]+(trans_tuples[i][1][0][0]*.5), crd[1]+(trans_tuples[i][1][0][1]*.5),crd[2]+(trans_tuples[i][1][0][2]*.5)])
    shifted_array = np.array(shifted_crds)
    IO.write_pdb(lig_prmtop, shifted_array, '/home/jim/Desktop/FFT_notebook/7.analysis/1AVZ_B:C/lowest_energy_pose%s.pdb'%i, "w")
    
#print(trans_lig_crd)
#trans_lig_crd_array = np.array(trans_lig_crd)
    

# for i, position in resampled_energies[0]:
#     min_index = np.where(resampled_energies[0] == resampled_energies[0].min())

In [ ]:
import prody

from os import listdir
from os.path import isfile, join

pose_path = '/home/jim/Desktop/FFT_notebook/7.analysis/1AVZ_B:C/'

pose_files = [f for f in listdir(pose_path) if isfile(join(pose_path, f))]

pdb_paths = []

for file in pose_files:
    if file[-4:] == '.pdb':
        path = '%s%s'%(pose_path,file)
        pdb_paths.append(path)

native = prody.parsePDB('/home/jim/Desktop/FFT_notebook/7.analysis/1AVZ_B:C/lowest_energy_pose0.pdb')
rmsds = []
for index, path in enumerate(pdb_paths):
    pose = prody.parsePDB(path)
    rmsds.append(prody.calcRMSD(native, pose))

In [ ]:
import netCDF4 as nc 
import matplotlib.pyplot as plt

lowest_energies = []
for energy, translation in trans_tuples:
    lowest_energies.append(energy)

# plot 1000 energies vs translations
fft_fn = '1EER_A:BC'
plt.scatter(rmsds, lowest_energies, s=20, marker='+')
plt.xlabel('RMSD (Å)')
plt.ylabel('Interaction Energy (kcal/mol)')
plt.savefig('%s.png'%fft_fn)
plt.show()

In [ ]:
import netCDF4 as nc
import numpy as np

grid_path = '/home/jim/Desktop/FFT_notebook/4.receptor_grid/1AVZ_B:C'
grid_nc = '%s/grid.nc'%grid_path


#parsing some of the netcdf variables from grid.nc
grid_variables = nc.Dataset(grid_nc, 'r').variables
counts = nc.Dataset(grid_nc, 'r').variables["counts"][:]
x = nc.Dataset(grid_nc, 'r').variables["x"][:]
y = nc.Dataset(grid_nc, 'r').variables["y"][:]
z = nc.Dataset(grid_nc, 'r').variables["z"][:]
electrostatic = nc.Dataset(grid_nc, 'r').variables["electrostatic"][:]
lja = nc.Dataset(grid_nc, 'r').variables["LJa"][:]
ljr = nc.Dataset(grid_nc, 'r').variables["LJr"][:]
occupancy = nc.Dataset(grid_nc, 'r').variables["occupancy"][:]
trans_crd = nc.Dataset(grid_nc, 'r').variables["trans_crd"][:]

#DELETE ME: print statements for inspecting the contents of grid.nc
#print(len(occupancy[200][:]))
#print(grid_variables)
kttc = []
for key in grid_variables:
    kttc.append(key)
#    print(grid_variables[key])
print(kttc)
print(counts)
#electrostatic grid is accessed by grid[x][y][z], to be confirmed. Alternative is grid[z][y][x]
#below code accesses electrostatic grid values for 3 points on the same axis
print(electrostatic[0][0][0],electrostatic[0][0][1],electrostatic[0][0][2])
#identifies occupied positions on the grid 
occupied_list = [i for i in range(len(occupancy[50][50])) if occupancy[50][50][i] == 1] 
          
# printing resultant list  
print ("New indices list : " + str(occupied_list)) 

#for i in res_list:
#    print(electrostatic[200][200][i])
    
#print(trans_crd)
#print(trans_crd[:])

def multiply_grids(electrostatic_grid, lja_grid, ljr_grid, occupancy_grid):
    grid_dot = np.empty((electrostatic_grid.shape[0], electrostatic_grid.shape[1], electrostatic_grid.shape[2]))
    for i in range(electrostatic_grid.shape[0]):
        for j in range(electrostatic_grid.shape[1]):
            for k in range(electrostatic_grid.shape[2]):
                grid_dot[i][j][k] = np.dot(electrostatic_grid[i][j][k], lja_grid[i][j][k])
                grid_dot[i][j][k] = np.dot(grid_dot[i][j][k], ljr_grid[i][j][k])
    return grid_dot

def get_meaningful_energies(energies):
    for i in range(energies.shape[0]):
        for j in range(energies.shape[1]):
            for k in range(energies.shape[2]):
                if energies[i][j][k] > 0.5:
                    print(energies[i][j][k])


#result = multiply_grids(electrostatic, lja, ljr, occupancy)
#get_meaningful_energies(result)
#print(result)
for i in trans_crd:
    if i[0] > 140:
        print(i)
    
native_trans = [i for i in range(len(trans_crd[:])) if trans_crd[i][0] == 140] 
print(len(trans_crd))
for point in occupied_list:
    print(trans_crd[point])
# get the center of mass of the crystallographic pose, then, get the center of mass for every
# translation of the native rotation and measure the distance between the two points. Select
# the translation with the nearest center of mass as the "native/crystallographic pose"



In [ ]:
#get the lowest energy translation for each rotation

from os import listdir
from os.path import isfile, join
import netCDF4 as nc
import numpy as np
import IO
import prody
import netCDF4 as nc 
import matplotlib.pyplot as plt

base_path = "/home/jim/Desktop/FFT_notebook/"
fft_path = base_path + "5.fft_sampling/"
amber_path = base_path + "1.amber/"
analysis_path = base_path + "7.analysis/"

#print([name for name in os.listdir("/home/jim/Desktop/FFT_notebook/5.fft_sampling") if os.path.isdir(name)])

fft_paths = [f for f in listdir(fft_path) if os.path.isdir(join(fft_path, f))]

print(fft_paths)
for path in fft_paths:
    fft_nc = fft_path + path + '/fft_sample.nc'
    print(fft_nc)
    lig_prmtop = amber_path + "%s/ligand.prmtop"%path
    resampled_energies = nc.Dataset(fft_nc, 'r').variables["resampled_energies"][:]
    resampled_trans_vectors = nc.Dataset(fft_nc, 'r').variables["resampled_trans_vectors"][:]
    lig_positions = nc.Dataset(fft_nc, 'r').variables["lig_positions"][:]
    
    trans_tuples = []
    for i, rotation in enumerate(resampled_energies):
        min_index = np.where(rotation == rotation.min())
        trans_tuples.append((rotation[min_index], resampled_trans_vectors[i][min_index]))

    if not os.path.exists(analysis_path + '%s/'%path):
        os.mkdir(analysis_path + '%s'%path)
    
    trans_lig_crd = []
    
    for i, position in enumerate(lig_positions):
        shifted_crds = []
        for j, crd in enumerate(position):
            shifted_crds.append([crd[0]+(trans_tuples[i][1][0][0]*.5), crd[1]+(trans_tuples[i][1][0][1]*.5),crd[2]+(trans_tuples[i][1][0][2]*.5)])
        shifted_array = np.array(shifted_crds)
        if not os.path.exists(analysis_path + '%s/lowest_energy_pose%s.pdb'%(path,i)):
            IO.write_pdb(lig_prmtop, shifted_array, analysis_path + '%s/lowest_energy_pose%s.pdb'%(path,i), "w")
    
    #read in the newly created pdb files for analysis TODO: make a trajectory you lazy bastard
    pose_path = analysis_path + '%s/'%path
    pose_files = [f for f in listdir(pose_path) if isfile(join(pose_path, f))]

    pdb_paths = []

    for file in pose_files:
        if file[-4:] == '.pdb':
            path_temp = '%s%s'%(pose_path,file)
            pdb_paths.append(path_temp)

    native = prody.parsePDB(analysis_path + path + '/lowest_energy_pose0.pdb')
    rmsds = []
    for index, p in enumerate(pdb_paths):
        pose = prody.parsePDB(p)
        rmsds.append(prody.calcRMSD(native, pose))
        
    #finally make a new list of the interaction energies and make some graphs
    lowest_energies = []
    for energy, translation in trans_tuples:
        lowest_energies.append(energy)

    # plot 1000 energies vs translations
    plt.scatter(rmsds, lowest_energies, s=20, marker='+')
    plt.xlabel('RMSD (Å)')
    plt.ylabel('Interaction Energy (kcal/mol)')
    plt.savefig(analysis_path + '%s.png'%path)
    plt.show()

In [ ]:
print(analysis_path + '%s.png'%path)
print(analysis_path)

In [ ]:
# get all translations less than 10% higher than the interaction energy of the native pose
# example: if the native pose interaction energy is -10 kcal/mol, get all interaction energies less than
# -9 kcal/mol. This is pretty arbitrary, so it might be helpful to develop a theoretical basis for the cutoff.

from os import listdir
from os.path import isfile, join, isdir, exists
import netCDF4 as nc
import numpy as np
import IO
import prody
import netCDF4 as nc 
import matplotlib.pyplot as plt

base_path = "/home/jim/Desktop/FFT_notebook/"
fft_path = base_path + "5.fft_sampling/"
amber_path = base_path + "1.amber/"
analysis_path = base_path + "7.analysis/"

#print([name for name in os.listdir("/home/jim/Desktop/FFT_notebook/5.fft_sampling") if os.path.isdir(name)])

fft_paths = [f for f in listdir(fft_path) if isdir(join(fft_path, f))]

print(fft_paths)
for path in fft_paths:
    fft_nc = fft_path + path + '/fft_sample.nc'
    print(fft_nc)
    lig_prmtop = amber_path + "%s/ligand.prmtop"%path
    resampled_energies = nc.Dataset(fft_nc, 'r').variables["resampled_energies"][:]
    resampled_trans_vectors = nc.Dataset(fft_nc, 'r').variables["resampled_trans_vectors"][:]
    lig_positions = nc.Dataset(fft_nc, 'r').variables["lig_positions"][:]
    
    trans_tuples = []
    native_energy = resampled_energies[0][0]
    for i, rotation in enumerate(resampled_energies):
        for j, pose in enumerate(rotation):
            if pose < (native_energy - (native_energy*0.01)):
                trans_tuples.append((pose, resampled_trans_vectors[i][j], i, j))

    if not exists(analysis_path + '%s/'%path):
        os.mkdir(analysis_path + '%s'%path)
    
    trans_lig_crd = []
    lowest_energies = []
    
    for number, (energy, translation, rot_index, pose_index) in enumerate(trans_tuples):
        shifted_crds = []
        origin_crd = lig_positions[rot_index]
        for i, crd in enumerate(origin_crd):
            shifted_crds.append([crd[0]+(translation[0]*.5), crd[1]+(translation[1]*.5),crd[2]+(translation[2]*.5)])
        shifted_array = np.array(shifted_crds)
        lowest_energies.append(energy)
        if not exists(analysis_path + '%s/lowest_energy_poses_test%s.pdb'%(path,number)):
            #print(rot_index, pose_index, energy)
            IO.write_pdb(lig_prmtop, shifted_array, analysis_path + '%s/lowest_energy_pose_test%s.pdb'%(path,number), "w")
    
    #read in the newly created pdb files for analysis TODO: make a trajectory you lazy bastard
    pose_path = analysis_path + '%s/'%path
    pose_files = [f for f in listdir(pose_path) if isfile(join(pose_path, f))]

    pdb_paths = []

    for file in pose_files:
        if file[-4:] == '.pdb':
            path_temp = '%s%s'%(pose_path,file)
            pdb_paths.append(path_temp)
    print(len(pdb_paths))
    native = prody.parsePDB(analysis_path + path + '/lowest_energy_pose_test0.pdb')
    rmsds = []
    for index, p in enumerate(pdb_paths):
        pose = prody.parsePDB(p)
        rmsds.append(prody.calcRMSD(native, pose))

        

    # plot 1000 energies vs translations
    print(len(lowest_energies))
    print(len(rmsds))
    plt.scatter(rmsds, lowest_energies, s=20, marker='+')
    plt.xlabel('RMSD (Å)')
    plt.ylabel('Interaction Energy (kcal/mol)')
    plt.savefig(analysis_path + '%s/RMSDvsIE.png'%path)
    plt.show()

In [ ]:
ref = prody.parsePDB('/home/jim/Desktop/FFT_notebook/7.analysis/1EER_A:BC/lowest_energy_pose_test0.pdb')
tar = prody.parsePDB('/home/jim/Desktop/FFT_notebook/7.analysis/1EER_A:BC/lowest_energy_pose_test1.pdb')
print(prody.calcRMSD(ref.getCoords(), tar.getCoords()))

print(ref.getCoords())

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import align, rms

ref = mda.Universe('/home/jim/Desktop/FFT_notebook/7.analysis/1EER_A:BC/lowest_energy_pose_test0.pdb')
mobile = mda.Universe('/home/jim/Desktop/FFT_notebook/7.analysis/1EER_A:BC/lowest_energy_pose_test1.pdb')

ref_sel = ref.select_atoms("protein")
mobile_sel = mobile.select_atoms("protein")

rms.rmsd(ref_sel.positions, mobile_sel.positions)